# Cinemática Directa usando Denavit-Hartenberg: Robot SCARA
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/fundamentos-robotica-python/blob/main/4-Cinematica-Directa/4-4-DH-Robot-SCARA.ipynb)

Oscar E. Ramos Ponce, Universidad de Ingeniería y Tecnología - UTEC

Fundamentos de Robótica

### 1.&nbsp;Cálculo Simbólico de la Cinemática Directa

Se utilizará sympy para encontrar la matriz de transformación homogéneas de Denavit-Hartenberg (convención estándar)

In [1]:
import sympy as sp                # Biblioteca para el cálculo simbólico

def sTdh(d, th, a, alpha):
    cth = sp.cos(th); sth = sp.sin(th)
    ca = sp.cos(alpha); sa = sp.sin(alpha)
    Tdh = sp.Matrix([[cth, -ca*sth,  sa*sth, a*cth],
                     [sth,  ca*cth, -sa*cth, a*sth],
                     [0,        sa,     ca,      d],
                     [0,         0,      0,      1]])
    return Tdh

In [2]:
# Variables simbólicas
q1, q2, q3, q4 = sp.symbols("q1 q2 q3 q4")
l1, l2, l3, l4 = sp.symbols("l1 l2 l3 l4")

# Transformaciones homogéneas
T01 = sTdh(    l1,    sp.pi+q1, l2,     0)
T12 = sTdh(     0, -sp.pi/2+q2, l3,     0)
T23 = sTdh(-l4+q3,           0,  0,     0)
T34 = sTdh(     0,  sp.pi/2+q4,  0, sp.pi)

# Transformación homogénea final
Tf = sp.simplify(T01*T12*T23*T34)

# Mostrar las transformaciones homogéneas (display funciona con IPython)
print("T01:"); display(T01)
print("T12:"); display(T12)
print("T23:"); display(T23)
print("T34:"); display(T34)
print("T04:"); display(Tf)

T01:


Matrix([
[-cos(q1),  sin(q1), 0, -l2*cos(q1)],
[-sin(q1), -cos(q1), 0, -l2*sin(q1)],
[       0,        0, 1,          l1],
[       0,        0, 0,           1]])

T12:


Matrix([
[ sin(q2), cos(q2), 0,  l3*sin(q2)],
[-cos(q2), sin(q2), 0, -l3*cos(q2)],
[       0,       0, 1,           0],
[       0,       0, 0,           1]])

T23:


Matrix([
[1, 0, 0,        0],
[0, 1, 0,        0],
[0, 0, 1, -l4 + q3],
[0, 0, 0,        1]])

T34:


Matrix([
[-sin(q4), cos(q4),  0, 0],
[ cos(q4), sin(q4),  0, 0],
[       0,       0, -1, 0],
[       0,       0,  0, 1]])

T04:


Matrix([
[-cos(q1 + q2 + q4), -sin(q1 + q2 + q4),  0, -l2*cos(q1) - l3*sin(q1 + q2)],
[-sin(q1 + q2 + q4),  cos(q1 + q2 + q4),  0, -l2*sin(q1) + l3*cos(q1 + q2)],
[                 0,                  0, -1,                  l1 - l4 + q3],
[                 0,                  0,  0,                             1]])

In [3]:
# Valor cuando todos los ángulos son cero
print("T04 cuando q=(0,0,0,0):")
Tf.subs([ (q1,0), (q2,0), (q3,0), (q4,0)])

T04 cuando q=(0,0,0,0):


Matrix([
[-1, 0,  0,     -l2],
[ 0, 1,  0,      l3],
[ 0, 0, -1, l1 - l4],
[ 0, 0,  0,       1]])

### 2.&nbsp;Cálculo Numérico de la Cinemática Directa

En este caso se utilizará numpy para el manejo de matrices y vectores

In [4]:
import numpy as np

def Tdh(d, th, a, alpha):
    cth = np.cos(th);    sth = np.sin(th)
    ca = np.cos(alpha);  sa = np.sin(alpha)
    Tdh = np.array([[cth, -ca*sth,  sa*sth, a*cth],
                    [sth,  ca*cth, -sa*cth, a*sth],
                    [0,        sa,     ca,      d],
                    [0,         0,      0,      1]])
    return Tdh

In [5]:
# Cinemática directa del robot
def cdirecta_scara(q, l1, l2, l3, l4):
    """ Retorna los sistemas de referencia de cada eslabón con respecto a la base
    """
    # Transformaciones homogéneas de DH
    T01 = Tdh(       l1,    np.pi+q[0], l2,     0)
    T12 = Tdh(        0, -np.pi/2+q[1], l3,     0)
    T23 = Tdh( -l4+q[2],             0,  0,     0)
    T34 = Tdh(        0,  np.pi/2+q[3],  0, np.pi)
    # Efector final con respecto a la base
    Tf = T01 @ T12 @ T23 @ T34
    return Tf

In [6]:
# Ejemplo de cálculo de la cinemática directa
l1 = 1.0                               # Longitud eslabón 1
l2 = 1.0                               # Longitud eslabón 2
l3 = 1.0                               # Longitud eslabón 3 
l4 = 0.5
q = [np.deg2rad(0), np.deg2rad(0), 0, np.deg2rad(0)]    # Valores articulares

# Cinemática directa
Te = cdirecta_scara(q, l1, l2, l3, l4)   # Cinemática directa

# Mostrar el resultado
print("Efector final con respecto a la base cuando q1={}, q2={}, q3={}, q4={}".format(np.rad2deg(q[0]), np.rad2deg(q[1]), 
                                                                                      q[2], np.rad2deg(q[3])))
print(np.round(Te,4))

Efector final con respecto a la base cuando q1=0.0, q2=0.0, q3=0, q4=0.0
[[-1.   0.   0.  -1. ]
 [ 0.   1.   0.   1. ]
 [ 0.   0.  -1.   0.5]
 [ 0.   0.   0.   1. ]]


### 3. Gráfico del Robot SCARA

Se utilizará una clase llamada `SerialRobot`, donde se definirá el robot utilizando los parámetros de Denavit-Hartenberg calculados. Usando esta clase se graficará el robot. Internamente, esta clase usa matplotlib para realizar el gráfico.

In [ ]:
# En Google Colab descomentar la siguiente línea y ejecutarla
#!wget https://raw.githubusercontent.com/oscar-ramos/fundamentos-robotica-python/main/4-Cinematica-Directa/serialrobot.py

In [1]:
from serialrobot import *

# Longitudes
l1 = 1.0; l2 = 1.0; l3 = 1.0; l4 = 0.5

# Parámetros de Denavit-Hartenberg para cada articulación
# Orden de los parámetros: d, th, a, alpha. Articulaciones: 'r' (revolución), 'p' (prismática)
L = [[ l1,    np.pi, l2,     0, 'r'],
     [  0, -np.pi/2, l3,     0, 'r'],
     [-l4,        0,  0,     0, 'p'],
     [  0,  np.pi/2,  0, np.pi, 'r']]

scara = SerialRobot(L, name='scara')
T = scara.fkine([0,0,0,0], verbose=False)
print(np.round(T,3))

[[-1.   0.   0.  -1. ]
 [ 0.   1.   0.   1. ]
 [ 0.   0.  -1.   0.5]
 [ 0.   0.   0.   1. ]]


Si se utiliza Google Colab, descomentar la siguiente línea. Luego reiniciar el entorno ("restart runtime") y ejecutar desde el inicio de la sección 3.

In [ ]:
# !pip install ipympl --quiet

In [ ]:
## Descomentar solo si se utiliza Google Colab
# from google.colab import output
# output.enable_custom_widget_manager()

In [2]:
%matplotlib widget

In [4]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection="3d")
alims = [[-2,2],[-2,2],[-0.2, 1.3]]    # Límites

scara.plot(ax, [0, 0, 0, 0], axlimits=alims, radius=0.1, color='r')

### 4.&nbsp;Animación del robot SCARA en Modo Local

Si se ha ejecutado la sección 3, donde se escoge el toolkit `widget` (usando '%matplotlib widget'), se debe primero reiniciar el entorno, ya que de otro modo no se podrá cambiar al toolkit `qt` necesario para la animación.

In [1]:
from serialrobot import *

# Longitudes
l1 = 1.0; l2 = 1.0; l3 = 1.0; l4 = 0.5

# Parámetros de Denavit-Hartenberg para cada articulación
# Orden de los parámetros: d, th, a, alpha. Articulaciones: 'r' (revolución), 'p' (prismática)
L = [[ l1,    np.pi, l2,     0, 'r'],
     [  0, -np.pi/2, l3,     0, 'r'],
     [-l4,        0,  0,     0, 'p'],
     [  0,  np.pi/2,  0, np.pi, 'r']]

# Creación del robot SCARA
scara = SerialRobot(L, name='scara')

In [2]:
# "qt" realiza el gráfico en una ventana nueva
%matplotlib qt

In [3]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
alims = [[-2,2],[-2,2],[-0.2, 1.3]]    # Límites

# Se abrirá una nueva ventana donde se visualizará el robot
for i in range(40):
    q = [np.deg2rad(i), np.deg2rad(0.5*i), 0.001*i, np.deg2rad(0.5*i)]    # En grados
    scara.plot(ax, q, axlimits=alims, radius=0.1, color='r')
    plt.pause(0.01)   

### 5.&nbsp;Animación del robot SCARA en Google Colab

En Google Colab descomentar y ejecutar la siguiente línea para instalar el paquete `celluloid`.

In [7]:
#!pip install celluloid --quiet

In [ ]:
from IPython.display import HTML
from celluloid import Camera

In [ ]:
from serialrobot import *

# Longitudes
l1 = 1.0; l2 = 1.0; l3 = 1.0; l4 = 0.5

# Parámetros de Denavit-Hartenberg para cada articulación
# Orden de los parámetros: d, th, a, alpha. Articulaciones: 'r' (revolución), 'p' (prismática)
L = [[ l1,    np.pi, l2,     0, 'r'],
     [  0, -np.pi/2, l3,     0, 'r'],
     [-l4,        0,  0,     0, 'p'],
     [  0,  np.pi/2,  0, np.pi, 'r']]

# Creación del robot SCARA
scara = SerialRobot(L, name='scara')

In [ ]:
#ax = plt.axes(projection='3d')
#fig = ax.figure
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
alims = [[-2,2],[-2,2],[-0.2, 1.3]]    # Límites

camera = Camera(fig)

for i in range(40):
    q = [np.deg2rad(i), np.deg2rad(0.5*i), 0.001*i, np.deg2rad(0.5*i)]    # En grados
    scara.plot(ax, q, axlimits=alims, radius=0.1, color='r')
    camera.snap()

plt.close()

In [ ]:
# Crear la animación
anim = camera.animate(interval = 40, repeat = True, repeat_delay = 500)

# Mostrar el resultado
HTML(anim.to_html5_video())